# Hey, thanks for being here!
## Capstone Project - Week 4
I know this is the final assignment but I don't feel too creative today, so I'll stick to the basic 2nd option, mentioned in the Assignment description:

*"In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it? Similarly, if a contractor is trying to start their own business, where would you recommend that they setup their office?"*

So I will pick Budapest, as that is my hometown and I will suggest neighborhoods for the entrepreneur. He/she wants to __start a gym__, because fitness is very trendy nowadays and I would not blame him to do that, because you can join several services like massage, dietary service, consulting on it.
Therefore I will map Budapest for possible starting points for his/her new franchise in the fitness industry.

The main idea is to query the Foursquare API for gyms in Budapest (or services like so), then pick the least dense area possible to minimize the concurrence. Of course that is not a quick lifehack or recipe for success but something to start with. Also we can review their average ratings, in that case, lower is better - I bet ppl are more open to new opportunities if the gym ratings are not that good.

## Capstone Project - Week 5
Let's get to work...

In [1]:
# Let's import everything we might need throughout our expedition :)

import requests 
import pandas as pd 
import numpy as np 
import random 
!pip install geopy
from geopy.geocoders import Nominatim 
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
import folium
import json
import urllib.request
import sys
!{sys.executable} -m pip install shapely
import shapely
from shapely.geometry import shape, Point

     |████████████████████████████████| 102kB 1.0MB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
     |████████████████████████████████| 1.5MB 1.0MB/s eta 0:00:01


In [2]:
# I'm sorry but I cannot display these for you
# But don't think I did not use them!

CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20190626'
LIMIT = 1000

Let's find Budapest's Longitude and Latitude values to start with.

In [3]:
address = 'Budapest'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

47.4983815 19.0404707


Let's construct the good old API URI.

In [4]:
# 14,5 km will suffice, we are not making money in the suburbs anyways..
radius = 14500

# We will search for the following categoryId, which is of the category Gym/Fitness.
categoryId = '4bf58dd8d48988d175941735'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, categoryId, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VPDI2D3XEB2I14NKL1EBV0RKHHK0Q4ZPPRJZIOXY2GFARGVV&client_secret=QJKHM15503UBRSR5FYC42D0EZOMSK5Q3WTIUVSSVKCMZHKLD&ll=47.4983815,19.0404707&v=20190626&categoryId=4bf58dd8d48988d175941735&radius=14500&limit=1000'

### Get them results!

In [5]:
results = requests.get(url).json()

We've got several hits to work with, let's get to prettify this a little bit.

In [6]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,4d52c5d39ffc236aafc830a7,Oxygen Wellness Naphegy,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1563965784,False,Naphegy u. 67.,47.491025,19.037900,"[{'label': 'display', 'lat': 47.49102528809318...",841,1016,HU,Budapest,Budapest,Magyarország,"[Budapest, Naphegy u. 67., 1016, Magyarország]",NaN,NaN,NaN
1,4cd992de7bb06dcb2209a9b2,Margitszigeti futópálya,"[{'id': '4bf58dd8d48988d106941735', 'name': 'T...",v-1563965784,False,Margitsziget,47.516065,19.044911,NaN,1996,1138,HU,Budapest,Budapest,Magyarország,"[Budapest, Margitsziget, 1138, Magyarország]",NaN,NaN,NaN
2,59bffd780c9f3164ce8be09d,CrossFit B'Bros 2,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1563965784,False,Márton utca 4.,47.476881,19.077628,"[{'label': 'display', 'lat': 47.47688050897686...",3679,1094,HU,Budapest,Budapest,Magyarország,"[Budapest, Márton utca 4. (Mester), 1094, Magy...",Mester,Budapest IX. kerülete,NaN
3,5b18fd3f345cbe002c363998,4% FiTNESS,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1563965784,False,Ráday utca 16.,47.487419,19.063551,"[{'label': 'display', 'lat': 47.48741894786818...",2121,1092,HU,Budapest IX. kerület,Budapest,Magyarország,"[Budapest IX. kerület, Ráday utca 16., 1092, M...",NaN,NaN,NaN
4,5b97e90f180b91002c09ea95,Massfitness,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1563965784,False,Bartók Béla út 61,47.476998,19.043483,"[{'label': 'display', 'lat': 47.47699823953018...",2391,1114,HU,Budapest,Budapest,Magyarország,"[Budapest, Bartók Béla út 61, 1114, Magyarország]",NaN,NaN,NaN


We've learned how to process these kind of tables, so let's do it:

In [7]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Oxygen Wellness Naphegy,Gym / Fitness Center,Naphegy u. 67.,47.491025,19.037900,"[{'label': 'display', 'lat': 47.49102528809318...",841,1016,HU,Budapest,Budapest,Magyarország,"[Budapest, Naphegy u. 67., 1016, Magyarország]",NaN,NaN,4d52c5d39ffc236aafc830a7
1,Margitszigeti futópálya,Track,Margitsziget,47.516065,19.044911,NaN,1996,1138,HU,Budapest,Budapest,Magyarország,"[Budapest, Margitsziget, 1138, Magyarország]",NaN,NaN,4cd992de7bb06dcb2209a9b2
2,CrossFit B'Bros 2,Gym / Fitness Center,Márton utca 4.,47.476881,19.077628,"[{'label': 'display', 'lat': 47.47688050897686...",3679,1094,HU,Budapest,Budapest,Magyarország,"[Budapest, Márton utca 4. (Mester), 1094, Magy...",Mester,Budapest IX. kerülete,59bffd780c9f3164ce8be09d
3,4% FiTNESS,Gym / Fitness Center,Ráday utca 16.,47.487419,19.063551,"[{'label': 'display', 'lat': 47.48741894786818...",2121,1092,HU,Budapest IX. kerület,Budapest,Magyarország,"[Budapest IX. kerület, Ráday utca 16., 1092, M...",NaN,NaN,5b18fd3f345cbe002c363998
4,Massfitness,Gym / Fitness Center,Bartók Béla út 61,47.476998,19.043483,"[{'label': 'display', 'lat': 47.47699823953018...",2391,1114,HU,Budapest,Budapest,Magyarország,"[Budapest, Bartók Béla út 61, 1114, Magyarország]",NaN,NaN,5b97e90f180b91002c09ea95
5,Highlander Club,Gym / Fitness Center,NaN,47.560819,18.948459,"[{'label': 'display', 'lat': 47.56081860877104...",9804,1029,HU,Budapest,Budapest,Magyarország,"[Budapest, 1029, Magyarország]",NaN,NaN,582b4d5dc4c9fa42dbd58f20
6,Nr1fitness,Gym,Aradi utca,47.506909,19.063011,"[{'label': 'display', 'lat': 47.50690917642486...",1942,1062,HU,Budapest,Budapest,Magyarország,"[Budapest, Aradi utca (Eötvös utca), 1062, Mag...",Eötvös utca,NaN,580cc71638fa626ebdf9b1c0
7,Life1 Corvin Wellness,Gym / Fitness Center,Futó u. 48-50.,47.485219,19.075089,"[{'label': 'display', 'lat': 47.48521905418784...",2987,1082,HU,Budapest,Budapest,Magyarország,"[Budapest, Futó u. 48-50., 1082, Magyarország]",NaN,NaN,4c5d8ec89b28d13a57d75970
8,BullsParkCrossfit,Gym,Soproni út 19.,47.468109,19.045225,"[{'label': 'display', 'lat': 47.46810900271648...",3388,1117,HU,Budapest,Budapest,Magyarország,"[Budapest, Soproni út 19., 1117, Magyarország]",NaN,Kelenföld,5a0f0c72cf72a0359c971dcd
9,Go/Active,Gym,NaN,47.498730,19.059408,"[{'label': 'display', 'lat': 47.49873001806004...",1424,1075,HU,Budapest VII. kerület,Budapest,Magyarország,"[Budapest VII. kerület, 1075, Magyarország]",NaN,NaN,5ab10862974617565d77d91c


In [8]:
# Checking shape:
dataframe_filtered.shape

(50, 16)

Well 50 is not really much, but not bad either. Poor developers only have that much. Nevermind, we can work with this.

### Well, we could visualize the results but what should we say for our client? Which districts should he/she look for buildings to rent?

I will visualize the results with the districts.
We need 2 things for this: 
- a dataset with all the districts
- a cloropleth map to visualize the results
For the sake of fun, I will make a new column in the 'dataframe_filtered' dataframe, to categorize them into districts.

But first, let me get our geojson file. 
There's geojson data available for every district but in separate files. let's download them from github.

In [9]:
# Make a list item for all the districts in Budapest

l = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']

In [10]:
# Make a list of URLS (because noone likes to copy-paste)
url_l = []
for i in l:
    url_l.append('https://raw.githubusercontent.com/integralvision/geo-data-hungary/master/GeoJSON/l40-district/0' + i + '-budapest-'+ i + '-kerulet.geojson')
url_l

['https://raw.githubusercontent.com/integralvision/geo-data-hungary/master/GeoJSON/l40-district/001-budapest-01-kerulet.geojson',
 'https://raw.githubusercontent.com/integralvision/geo-data-hungary/master/GeoJSON/l40-district/002-budapest-02-kerulet.geojson',
 'https://raw.githubusercontent.com/integralvision/geo-data-hungary/master/GeoJSON/l40-district/003-budapest-03-kerulet.geojson',
 'https://raw.githubusercontent.com/integralvision/geo-data-hungary/master/GeoJSON/l40-district/004-budapest-04-kerulet.geojson',
 'https://raw.githubusercontent.com/integralvision/geo-data-hungary/master/GeoJSON/l40-district/005-budapest-05-kerulet.geojson',
 'https://raw.githubusercontent.com/integralvision/geo-data-hungary/master/GeoJSON/l40-district/006-budapest-06-kerulet.geojson',
 'https://raw.githubusercontent.com/integralvision/geo-data-hungary/master/GeoJSON/l40-district/007-budapest-07-kerulet.geojson',
 'https://raw.githubusercontent.com/integralvision/geo-data-hungary/master/GeoJSON/l40-dis

In [11]:
# let's aggregate them into 1 geojson file..
k = 0
data_d = {}
data = {}

for i in url_l:
    with urllib.request.urlopen(i) as url:
        data = {}
        data = json.loads(url.read().decode())
        if k == 0:
            data_d = data
            k += 1
        else:
            data_d['features'].append(data['features'][0])

# ...and save it in a local 'districts.json' file
with open('districts.json', 'w') as fp:
     json.dump(data_d, fp)

### We have the geojson data, but before we proceed to make the cloropleth map:
We talked about the different density of districts, it would be very good if we could visualize it with all the gyms and make a column for the ratio of potential gym users per $km^2$.
Like $\frac{No of ppl}{gym*km^2}$. So let's not just visualize it, but add them to the dataframe!

In [12]:
# Let's get the density data of each district in the meantime. There is some 2016 data available on wikipedia (https://en.wikipedia.org/wiki/List_of_districts_in_Budapest)

density ='''District	Population	Area	Density
I. kerület	25196	3.41	7388.8
II. kerület	89903	36.34	2473.9
III. kerület	130415	39.70	3285.0
IV. kerület	101558	18.82	5396.2
V. kerület	26284	2.59	10148.2
VI. kerület	38504	2.38	16178.1
VII. kerület	53381	2.09	25541.1
VIII. kerület	76811	6.85	11213.2
IX. kerület	59056	12.53	4713.1
X. kerület	78414	32.49	2413.4
XI. kerület	151812	33.49	4533.0
XII. kerület	58171	26.67	2181.1
XIII. kerület	120256	13.43	8954.2
XIV. kerület	124841	18.13	6885.8
XV. kerület	80573	26.94	2990.8
XVI. kerület	73486	33.51	2192.9
XVII. kerület	87793	54.82	1601.4
XVIII. kerület	101738	38.60	2635.6
XIX. kerület	60731	9.38	6474.5
XX. kerület	65321	12.19	5385.5
XXI. kerület	76911	25.75	2985.8
XXII. kerület	54611	34.25	1594.4
XXIII. kerület	23641	40.77	579.8'''

density_l = []
for i in density.split('\n'):
    density_l.append(i.split('\t'))
df_distd = pd.DataFrame(density_l[1:], columns=density_l[0])
df_distd.head()

,District,Population,Area,Density
0,I. kerület,25196,3.41,7388.8
1,II. kerület,89903,36.34,2473.9
2,III. kerület,130415,39.70,3285.0
3,IV. kerület,101558,18.82,5396.2
4,V. kerület,26284,2.59,10148.2


The above dataframe looks good. Let's use the density column.

In [13]:

with open('districts.json', 'r') as myfile:
    js = json.load(myfile)
d_list = []
for district in js['features']:
    d_prop = []
    d_prop.append(district['properties']['name']) # .replace('kerület', 'District').replace('Budapest ', '') ->> sorry, I would replace it to make it more readable, but i would make trouble for myself when choroplething...
    d_prop.append(shape(district['geometry']).centroid.coords[0][1])
    d_prop.append(shape(district['geometry']).centroid.coords[0][0])
    d_list.append(d_prop)

dist_columns=['District', 'Center_Lat', 'Center_Lng']
pd_dist = pd.DataFrame(d_list, columns=dist_columns)
pd_dist['Density'] = df_distd['Density'].astype(float)
pd_dist.head()

,District,Center_Lat,Center_Lng,Density
0,Budapest I. kerület,47.496961,19.035772,7388.8
1,Budapest II. kerület,47.542218,18.979611,2473.9
2,Budapest III. kerület,47.569975,19.033603,3285.0
3,Budapest IV. kerület,47.579622,19.095804,5396.2
4,Budapest V. kerület,47.500354,19.050677,10148.2


Before we start making maps, let's set the threshold scale (believe me, I tried it, automatic does not work...)

In [14]:
# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(pd_dist['Density'].min(),
                              pd_dist['Density'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration
threshold_scale

[579, 5572, 10564, 15556, 20548, 25542]

In [15]:
# Make Cloropleth map with the help of our geojson file
from folium.features import DivIcon

venues_map = folium.Map(location=[latitude, longitude], zoom_start=11)
districts_geo = r'districts.json'
venues_map.choropleth(
    geo_data=districts_geo,
    name='Choromonomomo',
    data = pd_dist,
    columns = ['District', 'Density'],
    key_on = 'feature.properties.name',
    threshold_scale=threshold_scale,
    fill_color = 'YlOrRd',
    fill_opacity = 0.65,
    line_opacity = 0.2,
    legend_name = 'Population density of Budapest'
)

# Adding the district names
for p in pd_dist.iterrows():
    folium.map.Marker(
        [p[1][1], p[1][2]],
        icon=DivIcon(
            icon_anchor=(0,0),
            html='<div style="font-size:1.0em;color:black">%s</div>' % p[1][0],
            )
        ).add_to(venues_map)

# Let's take a look at our work, shall we?
venues_map

That's not bad. 
Let us continue with the integration of the new population density data, shall we?
First, let us decide where do gyms belong. For a quick reminder, let's see our *dataframe_filtered* dataframe.


In [16]:
dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Oxygen Wellness Naphegy,Gym / Fitness Center,Naphegy u. 67.,47.491025,19.037900,"[{'label': 'display', 'lat': 47.49102528809318...",841,1016,HU,Budapest,Budapest,Magyarország,"[Budapest, Naphegy u. 67., 1016, Magyarország]",NaN,NaN,4d52c5d39ffc236aafc830a7
1,Margitszigeti futópálya,Track,Margitsziget,47.516065,19.044911,NaN,1996,1138,HU,Budapest,Budapest,Magyarország,"[Budapest, Margitsziget, 1138, Magyarország]",NaN,NaN,4cd992de7bb06dcb2209a9b2
2,CrossFit B'Bros 2,Gym / Fitness Center,Márton utca 4.,47.476881,19.077628,"[{'label': 'display', 'lat': 47.47688050897686...",3679,1094,HU,Budapest,Budapest,Magyarország,"[Budapest, Márton utca 4. (Mester), 1094, Magy...",Mester,Budapest IX. kerülete,59bffd780c9f3164ce8be09d
3,4% FiTNESS,Gym / Fitness Center,Ráday utca 16.,47.487419,19.063551,"[{'label': 'display', 'lat': 47.48741894786818...",2121,1092,HU,Budapest IX. kerület,Budapest,Magyarország,"[Budapest IX. kerület, Ráday utca 16., 1092, M...",NaN,NaN,5b18fd3f345cbe002c363998
4,Massfitness,Gym / Fitness Center,Bartók Béla út 61,47.476998,19.043483,"[{'label': 'display', 'lat': 47.47699823953018...",2391,1114,HU,Budapest,Budapest,Magyarország,"[Budapest, Bartók Béla út 61, 1114, Magyarország]",NaN,NaN,5b97e90f180b91002c09ea95


We have a neighborhood column, but that contains unreliable data. Let's decide based on the the latitude and longitude values which district they belong to, and then redefine the values.

In [17]:
lst_ds = []
with open('districts.json', 'r') as myfile:
    js = json.load(myfile)

# iterate through rows and find each point in the districts' polygons
for row in dataframe_filtered.iterrows():
    point = Point(row[1]['lng'], row[1]['lat'])
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            lst_ds.append(feature['properties']['name'])

# Rewrite values
dataframe_filtered['neighborhood'] = np.array(lst_ds)
dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Oxygen Wellness Naphegy,Gym / Fitness Center,Naphegy u. 67.,47.491025,19.037900,"[{'label': 'display', 'lat': 47.49102528809318...",841,1016,HU,Budapest,Budapest,Magyarország,"[Budapest, Naphegy u. 67., 1016, Magyarország]",NaN,Budapest I. kerület,4d52c5d39ffc236aafc830a7
1,Margitszigeti futópálya,Track,Margitsziget,47.516065,19.044911,NaN,1996,1138,HU,Budapest,Budapest,Magyarország,"[Budapest, Margitsziget, 1138, Magyarország]",NaN,Budapest XIII. kerület,4cd992de7bb06dcb2209a9b2
2,CrossFit B'Bros 2,Gym / Fitness Center,Márton utca 4.,47.476881,19.077628,"[{'label': 'display', 'lat': 47.47688050897686...",3679,1094,HU,Budapest,Budapest,Magyarország,"[Budapest, Márton utca 4. (Mester), 1094, Magy...",Mester,Budapest IX. kerület,59bffd780c9f3164ce8be09d
3,4% FiTNESS,Gym / Fitness Center,Ráday utca 16.,47.487419,19.063551,"[{'label': 'display', 'lat': 47.48741894786818...",2121,1092,HU,Budapest IX. kerület,Budapest,Magyarország,"[Budapest IX. kerület, Ráday utca 16., 1092, M...",NaN,Budapest IX. kerület,5b18fd3f345cbe002c363998
4,Massfitness,Gym / Fitness Center,Bartók Béla út 61,47.476998,19.043483,"[{'label': 'display', 'lat': 47.47699823953018...",2391,1114,HU,Budapest,Budapest,Magyarország,"[Budapest, Bartók Béla út 61, 1114, Magyarország]",NaN,Budapest XI. kerület,5b97e90f180b91002c09ea95


We can visualize this on our nice choroplath map.

In [18]:
# add the Gyms as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue'
    ).add_to(venues_map)

# display map
venues_map

One more thing we shall not forget about and that is **rating**. Let's query the rating for each Gym. We will save the responses to hold onto that precious data.

In [ ]:
import os

ids = []
for row in dataframe_filtered.iterrows():
    ids.append(row[1]['id'])

for _id in ids:
    venue_id = _id
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    fn = 'places/' + _id + '.json'
    os.makedirs(os.path.dirname(fn), exist_ok=True)
    with open('places/' + _id + '.json', 'w+') as fp:
        json.dump(result, fp)

Extract the ratings..

In [19]:
# Iterate through the directory and store the filenames into a list
i = 0
fileList = []
for filename in os.listdir('places'):
    if(filename[-4:]) == 'json':
        fileList.append(filename)

# Extract the ratings
# ..and take good care of the ones that has no rating.

rating_list = []
for file in fileList:
    with open(('places/'+ file), 'r') as myFile:
        js = json.load(myFile)
        if 'rating' not in js['response']['venue']:
            rating_list.append([js['response']['venue']['id'], np.nan])
        else:
            rating_list.append([js['response']['venue']['id'], js['response']['venue']['rating']])

# Store it in a dataframe
df_ratings = pd.DataFrame(rating_list, columns=['id', 'rating'])
df_ratings.head()

,id,rating
0,59bffd780c9f3164ce8be09d,NaN
1,580cc71638fa626ebdf9b1c0,8.6
2,53a85b9c498e26d9ca5e3d1f,9.0
3,5ab10862974617565d77d91c,NaN
4,586f9d80d4ab7350e1ccdf4d,8.7


In [20]:
df_newMerged = dataframe_filtered.merge(df_ratings, on='id')
df_newMerged.drop(columns=['address', 'distance', 'postalCode', 'cc', 'city', 'state', 'country', 'formattedAddress', 'crossStreet', 'labeledLatLngs'], inplace=True)

# let's get rid of the NaN rating by passing in an average 5.0
df_newMerged.fillna(5.0, inplace=True)
df_newMerged.head()

,name,categories,lat,lng,neighborhood,id,rating
0,Oxygen Wellness Naphegy,Gym / Fitness Center,47.491025,19.037900,Budapest I. kerület,4d52c5d39ffc236aafc830a7,8.9
1,Margitszigeti futópálya,Track,47.516065,19.044911,Budapest XIII. kerület,4cd992de7bb06dcb2209a9b2,9.1
2,CrossFit B'Bros 2,Gym / Fitness Center,47.476881,19.077628,Budapest IX. kerület,59bffd780c9f3164ce8be09d,5.0
3,4% FiTNESS,Gym / Fitness Center,47.487419,19.063551,Budapest IX. kerület,5b18fd3f345cbe002c363998,5.0
4,Massfitness,Gym / Fitness Center,47.476998,19.043483,Budapest XI. kerület,5b97e90f180b91002c09ea95,5.0


But let's get back to our data. Let's count the gyms in the different districts. Our *dataframe_filtered* dataframe is a bit too crowded, so let's make a brand new.

In [21]:
# A little modification
pd_dist.set_index('District', inplace=True)

Let's extract the average rating of gyms by district!

In [22]:
grouped = df_newMerged[['name', 'neighborhood', 'rating']].groupby('neighborhood').mean()
grouped.rename(columns={'rating':'averageRating'}, inplace=True)
grouped.head()

,averageRating
neighborhood,
Budapest I. kerület,8.55
Budapest II. kerület,7.00
Budapest III. kerület,7.84
Budapest IV. kerület,8.60
Budapest IX. kerület,6.95


Remember the District table with the population density? Our real goal is to find the __highest population__ density area with the __least number of gyms__ with the __lowest ratings__. So let's concatenate the two dataframes and see what we can do.

In [23]:
# Our final working dataframe
#

df_total = pd_dist.join(grouped[['averageRating']])

# Once again we fill the NaN with 5.0
df_total.fillna(5.0, inplace=True)
df_total.head()

,Center_Lat,Center_Lng,Density,averageRating
District,,,,
Budapest I. kerület,47.496961,19.035772,7388.8,8.55
Budapest II. kerület,47.542218,18.979611,2473.9,7.00
Budapest III. kerület,47.569975,19.033603,3285.0,7.84
Budapest IV. kerület,47.579622,19.095804,5396.2,8.60
Budapest V. kerület,47.500354,19.050677,10148.2,5.00


Let's create a new column with the explicit measuring unit. And that is: density/average_rating. The bigger number is the better, because we want high density with low ratings, so the new gym will have a great audience but little competition around.


In [24]:
df_total['density/avgRating'] = df_total['Density']/df_total['averageRating']

In [27]:
df_total.sort_values(by='density/avgRating', ascending=False)

,Center_Lat,Center_Lng,Density,averageRating,density/avgRating
District,,,,,
Budapest VII. kerület,47.502363,19.073516,25541.1,5.000000,5108.220000
Budapest VI. kerület,47.509549,19.065281,16178.1,7.925000,2041.400631
Budapest V. kerület,47.500354,19.050677,10148.2,5.000000,2029.640000
Budapest VIII. kerület,47.490601,19.088423,11213.2,7.650000,1465.777778
Budapest XIX. kerület,47.449383,19.143937,6474.5,5.000000,1294.900000
Budapest XIII. kerület,47.535532,19.068858,8954.2,8.166667,1096.432653
Budapest XX. kerület,47.433377,19.120929,5385.5,5.000000,1077.100000
Budapest XIV. kerület,47.518108,19.113713,6885.8,7.820000,880.537084
Budapest I. kerület,47.496961,19.035772,7388.8,8.550000,864.187135


### And there we have it. We can now suggest some starting location to look around. 

#### Although...
..we have some limitations though, I'm sure the reader is not surprised. There are more than 50 gyms around Budapest, so we can only make a very limited suggestions. 
We also cheated a little bit with the NaN filling, because we assume that the no-rating gyms have only 5.0 rating (it may be that the place is so good that the customers are usually get lost in working out and totally forget about their phones)

So I would not suggest for the reader to make big conclusions after reading my document, nevertheless, starting a new gym at the 7th district of Budapest. :)
#### BUT!
If we would really want to use the 4square API and make some market-research about the gyms of Budapest, would invest into expensive API calls and such, the method would be almost sufficient, to draw preliminary conclusions.


## Big thanks for reading and rating my submission! Here's a happy fluffy duck to express my gratitude.

<img src="https://proxy.duckduckgo.com/iu/?u=http%3A%2F%2Fi.ebayimg.com%2Fimages%2Fi%2F331303150488-0-1%2Fs-l1000.jpg&f=1" style="width:500px">